In [ ]:
import tensorflow as tf
import pandas as pd
import platform
import numpy as np

In [ ]:
if platform.system() == "Linux" or platform.system() == "Java":
    train = pd.read_csv('./data/train.csv')
    test = pd.read_csv('./data/test.csv')
else: 
    train = pd.read_csv('.\data\train.csv')
    test = pd.read_csv('.\data\test.csv')

In [ ]:
train.head()

In [ ]:
lens = train.comment_text.str.len()

lens.mean(), lens.std(), lens.max()

In [ ]:
hist = lens.hist(bins=100)
hist.set_xlim(0, 1000)
start, end = hist.get_xlim()
hist.xaxis.set_ticks(np.arange(start, end, 100));

In [ ]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
train['none'] = 1-train[label_cols].max(axis=1)
train.describe()

In [ ]:
COMMENT = 'comment_text'
train[COMMENT].fillna('unknown', inplace=True)
test[COMMENT].fillna('unknown', inplace=True)

In [ ]:
import re, string
re_token = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): return re_tok.sub(r' \1 ', s).split()

'__main__'